In [1]:
from ortools.linear_solver import pywraplp
from ortools.linear_solver import pywraplp
import numpy as np
from itertools import combinations
import pandas as pd
import numpy as np

In [2]:
dose_matrix = pd.read_csv('DoseMatrix.csv')

In [3]:
dose_matrix =np.array(dose_matrix)

In [4]:
CTV_index=[167, 168, 169, 170, 171, 172, 173,
      186, 187, 188, 189, 190, 191, 192, 193, 194,
      206, 207, 208, 209, 210, 211, 212, 213, 214, 
           227, 228, 229, 230, 231, 232, 233]

bladder_index=[87, 88, 89, 90, 91, 92, 93, 
         106, 107, 108, 109, 110, 111, 112, 113, 114, 
         126, 127, 128, 129, 130, 131, 132, 133, 134, 
             147, 148, 149, 150, 151, 152, 153]

Rectal_index=[248, 249, 250, 251, 252, 
        267, 268, 269, 270, 271, 272, 273, 
        287, 288, 289, 290, 291, 292, 293, 
              308, 309, 310, 311, 312]

Un_index=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 
             21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 
             41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 
             61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 
             81, 82, 83, 84, 85, 86, 94, 95, 96, 97, 98, 99, 100, 101, 104, 105, 115, 118, 119, 120,
             125, 139, 140, 145, 146, 154, 159, 160, 165, 166, 174, 179, 180, 185, 199, 200, 205, 219, 220, 
             225, 226, 234, 239, 240, 241, 244, 245, 246, 247, 253, 254, 255, 258, 259, 260, 261, 262, 263, 
             264, 265, 266, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 294, 295, 296, 
             297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 313, 314, 315, 316, 317, 318, 319, 320, 
             321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 
             340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 
             359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 
             378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 
             397, 398, 399]

Left_index=[116, 117, 
       135, 136, 137, 138, 
       155, 156, 157, 158, 
       175, 176, 177, 178,
       195, 196, 197, 198, 
       215, 216, 217, 218, 
       235, 236, 237, 238, 
            256, 257]

Right_index=[101, 102, 
        120, 121, 122, 123, 
        140, 141, 142, 143, 
        160, 161, 162, 163, 
        180, 181, 182, 183, 
        200, 201, 202, 203, 
        220, 221, 222, 223, 
             241, 242]

In [5]:
len(CTV_index)+len(bladder_index)+len(Rectal_index)+len(Un_index)+len(Left_index)+len(Right_index)

400

In [6]:
solver = pywraplp.Solver('Final Project',
                         pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
objective=solver.Objective() 
objective.SetMinimization()

# Decision Variables
beamlets = []
for i in range(60):
    beamlets.append(solver.NumVar(0,solver.infinity(), 'beamlets[%d]' % i))
voxels = []    
for i in range(400):
    voxels.append(solver.NumVar(0,solver.infinity(),'voxels[%d]' % i))

    
#Constraints
voxel_constraints = [0 for _ in range(400)]
for i in range(400):
    voxel_constraints[i] = solver.Constraint(0,0)
    for j in range(60):
        voxel_constraints[i].SetCoefficient(beamlets[j],dose_matrix[i,j+1])
    voxel_constraints[i].SetCoefficient(voxels[i], -1)
        
######################################################## CTV#################################################

# every voxel receives a uniform dose of 82.8       ( = 82.8)
z_p = [0 for _ in range(len(CTV_index))]
z_n = [0 for _ in range(len(CTV_index))]

for i in range(len(CTV_index)):
    z_p[i] = solver.NumVar(0, 82.8*0.06, "Z plus CTV %d" %i)
    z_n[i] = solver.NumVar(0, 82.8*0.05, "Z minus CTV %d" %i)
     
CTV_constraint = [0 for _ in range(len(CTV_index))]
for i in range(len(CTV_index)):
    CTV_constraint[i] = solver.Constraint(82.8,82.8)
    
    for j in range(60):
        CTV_constraint[i].SetCoefficient(beamlets[j],dose_matrix[CTV_index[i],j+1])
    CTV_constraint[i].SetCoefficient(z_p[i],1)
    CTV_constraint[i].SetCoefficient(z_n[i],-1)

    
    
############################################ bladder ####################################################

# # max dose to a voxel 81 ( < = 81)
z_b_1 = [0 for _ in range(len(bladder_index))]

for i in range(len(bladder_index)):
    z_b_1[i] = solver.NumVar(0, solver.infinity(), "z bladder #1 %d" %i)
    

bladder_constraint_1 = [0 for _ in range(len(bladder_index))]
for i in range(len(bladder_index)):
    bladder_constraint_1[i] = solver.Constraint(-solver.infinity(),81)
    for j in range(60):
        bladder_constraint_1[i].SetCoefficient(beamlets[j],dose_matrix[bladder_index[i],j+1])
    bladder_constraint_1[i].SetCoefficient(z_b_1[i],-1)


# Average dose should be <= 50     ( <= 50)

z_b_2 = solver.NumVar(0, solver.infinity(), "z bladder #2 %d" %i)
bladder_constraint_2 = solver.Constraint(-solver.infinity(), 50)
bladder_constraint_2.SetCoefficient(z_b_2, -1)

for i in bladder_index:
    bladder_constraint_2.SetCoefficient(voxels[i],1/len(bladder_index))

        
    
# at most 10% of the bladder should receive a dose >65          ( >65)
y = [0 for _ in range(len(bladder_index))]
z_y = solver.IntVar(0,1, "z y constraint bladder")

for i in range(len(bladder_index)):
     y[i] = solver.IntVar(0,1,"y bladder %d" %i)

bladder_constraint_3 = [0 for _ in range(len(bladder_index))]
for i in range(len(bladder_index)):
    bladder_constraint_3[i] = solver.Constraint(-solver.infinity(),65)
    for j in range(60):
        bladder_constraint_3[i].SetCoefficient(beamlets[j],dose_matrix[bladder_index[i],j+1])
    bladder_constraint_3[i].SetCoefficient(y[i],-100)

bladder_constraint_3b = solver.Constraint(-solver.infinity(),0.1*len(bladder_index))
for j in range(len(bladder_index)):
     bladder_constraint_3b.SetCoefficient(y[j],1)
bladder_constraint_3b.SetCoefficient(z_y, -1)

        
        

        
# ####################### Rectal########################################

#max dose to a voxel: 79.2              ( < = 79.2)

z_r_1 = [0 for _ in range(len(Rectal_index))]

for i in range(len(Rectal_index)):
    z_r_1[i] = solver.NumVar(0, solver.infinity(), "z rectal #1 %d" %i)
    

Rectal_constraint_1 = [0 for _ in range(len(Rectal_index))]
for i in range(len(Rectal_index)):
    Rectal_constraint_1[i] = solver.Constraint(-solver.infinity(),79.2)
    for j in range(60):
        Rectal_constraint_1[i].SetCoefficient(beamlets[j],dose_matrix[Rectal_index[i],j+1])
    Rectal_constraint_1[i].SetCoefficient(z_r_1[i],-1)

    
    
# #average dose should be <= 40

z_r_2 = solver.NumVar(0, solver.infinity(), "z rectal #2 %d" %i)
Rectal_constraint_2 = solver.Constraint(-solver.infinity(), 40)
Rectal_constraint_2.SetCoefficient(z_r_2, -1.0)

for i in Rectal_index:
    Rectal_constraint_2.SetCoefficient(voxels[i],1/len(Rectal_index))



# ############################## Unspecified ############################

# max dose to a voxel: 72         ( < =72)

z_u_1 = [0 for _ in range(len(Un_index))]

for i in range(len(Un_index)):
    z_u_1[i] = solver.NumVar(0, solver.infinity(), "z Unspecified #1 %d" %i)
    

Un_constraint_1 = [0 for _ in range(len(Un_index))]
for i in range(len(Un_index)):
    Un_constraint_1[i] = solver.Constraint(-solver.infinity(),72)
    for j in range(60):
        Un_constraint_1[i].SetCoefficient(beamlets[j],dose_matrix[Un_index[i],j+1])
    Un_constraint_1[i].SetCoefficient(z_u_1[i],-1)


#############################Left femur head ##########################

#max dose to a voxel: 50            ( <= 50)
  
z_l_1 = [0 for _ in range(len(Left_index))]

for i in range(len(Left_index)):
    z_l_1[i] = solver.NumVar(0, solver.infinity(), "z left femur #1 %d" %i)
    

left_constraint_1 = [0 for _ in range(len(Left_index))]
for i in range(len(Left_index)):
    left_constraint_1[i] = solver.Constraint(-solver.infinity(),50)
    for j in range(60):
        left_constraint_1[i].SetCoefficient(beamlets[j],dose_matrix[Left_index[i],j+1])
    left_constraint_1[i].SetCoefficient(z_l_1[i],-1)    
        

        
# at most 10% of the left femur should receive a dose > 40 
y_left = [0 for _ in range(len(Left_index))]
z_y_left = solver.IntVar(0,2, "z y constraint left femur")

for i in range(len(Left_index)):
     y_left[i] = solver.IntVar(0,1,"y left femur %d" %i)

Left_constraint_3 = [0 for _ in range(len(Left_index))]
for i in range(len(Left_index)):
    Left_constraint_3[i] = solver.Constraint(-solver.infinity(),40)
    for j in range(60):
        Left_constraint_3[i].SetCoefficient(beamlets[j],dose_matrix[Left_index[i],j+1])
    Left_constraint_3[i].SetCoefficient(y_left[i],-100)

Left_constraint_3b = solver.Constraint(-solver.infinity(),0.15*len(Left_index))
for j in range(len(Left_index)):
     Left_constraint_3b.SetCoefficient(y_left[j],1)
Left_constraint_3b.SetCoefficient(z_y_left,-1)


############################# right femur head ##########################


#max dose to a voxel: 50               ( <= 50 )
z_rl_1 = [0 for _ in range(len(Right_index))]

for i in range(len(Right_index)):
    z_rl_1[i] = solver.NumVar(0, solver.infinity(), "z right femur #1 %d" %i)
    

Right_constraint_1 = [0 for _ in range(len(Right_index))]
for i in range(len(Right_index)):
    Right_constraint_1[i] = solver.Constraint(-solver.infinity(),50)
    for j in range(60):
        Right_constraint_1[i].SetCoefficient(beamlets[j],dose_matrix[Right_index[i],j+1])
    Right_constraint_1[i].SetCoefficient(z_rl_1[i],-1)    
        

        
# at most 10% of the right femur should receive a dose > 40 
y_right = [0 for _ in range(len(Right_index))]
z_y_right = solver.IntVar(0,2, "z y constraint right femur")

for i in range(len(Right_index)):
     y_right[i] = solver.IntVar(0,1,"y right femur %d" %i)

Right_constraint_3 = [0 for _ in range(len(Right_index))]
for i in range(len(Right_index)):
    
    # <= 40
    Right_constraint_3[i] = solver.Constraint(-solver.infinity(),40)
    for j in range(60):
        Right_constraint_3[i].SetCoefficient(beamlets[j],dose_matrix[Right_index[i],j+1])
    # Big M constraint
    Right_constraint_3[i].SetCoefficient(y_right[i],-100)

Right_constraint_3b = solver.Constraint(-solver.infinity(),0.15*len(Right_index))
for j in range(len(Right_index)):
     Right_constraint_3b.SetCoefficient(y_right[j],1)
Right_constraint_3b.SetCoefficient(z_y_right, -1)
        
        
########################## Objective Function ###########################

# CTV
for i in range(len(z_p)):
    objective.SetCoefficient(z_p[i],2000)
    objective.SetCoefficient(z_n[i],2000)

# Bladders
for i in range(len(z_b_1)):
    objective.SetCoefficient(z_b_1[i],1)

objective.SetCoefficient(z_b_2,1)

# Rectal Solids
for i in range(len(z_r_1)):
    objective.SetCoefficient(z_r_1[i],1000)
    
objective.SetCoefficient(z_r_2,1)

#Unspecified
for i in range(len(z_u_1)):
    objective.SetCoefficient(z_u_1[i],1)

# left femur head
for i in range(len(z_l_1)):
    objective.SetCoefficient(z_l_1[i],10000)

#right femur head
for i in range(len(z_rl_1)):
    objective.SetCoefficient(z_rl_1[i],1)

# bladder, left femur, right femur's at most constraint
objective.SetCoefficient(z_y,1)
objective.SetCoefficient(z_y_left,2000)
objective.SetCoefficient(z_y_right,1)

status = solver.Solve()

if status == solver.OPTIMAL:
    print('Problem solved in %f milliseconds' %solver.wall_time())
elif status == solver.FEASIBLE:
    print('Solver claims feasibility but not optimality')
else:
    print('Solver ran to completion but did not find an optimal solution')

print('weighted slack total',objective.Value()) 


#print(bladder_constraint_2.dual_value())

#print(solver.ExportModelAsLpFormat(False).replace('\\', '').replace(',_', ','), sep='\n')

Problem solved in 9763.000000 milliseconds
weighted slack total 377761.932016873


In [7]:
print(bladder_constraint_2.dual_value())

0.0


In [8]:
#print(solver.ExportModelAsLpFormat(False).replace('\\', '').replace(',_', ','), sep='\n')

## All Voxels

In [9]:
All_voxels = np.zeros((400)).astype(np.float)
for i in range(400):
    All_voxels[i] = voxels[i].solution_value()
#print(All_voxels.reshape(20,20))
All_voxels = pd.DataFrame(All_voxels.reshape((20,20)), index=[i for i in range(1,21)],
                          columns = [i for i in range(1,21)]).round(2)
All_voxels

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
1,64.61,46.55,30.30,23.19,24.23,25.26,19.88,11.23,5.91,5.64,7.81,10.61,15.42,22.12,26.16,25.25,22.75,21.46,20.99,21.08
2,66.94,59.32,44.38,33.18,30.62,30.38,24.51,14.84,8.26,7.18,9.29,12.90,19.42,28.25,33.54,32.01,27.44,24.07,22.80,23.52
3,55.34,61.29,55.79,45.89,39.93,36.63,29.54,19.37,12.10,10.37,12.59,17.42,25.96,36.36,41.37,37.60,30.45,25.83,24.91,26.18
4,36.46,50.68,57.52,56.28,51.65,45.25,35.26,24.22,17.31,16.18,19.27,25.38,34.96,45.06,47.94,41.32,32.37,27.68,27.15,27.13
5,19.19,33.50,47.90,58.02,61.38,56.00,42.92,29.76,23.72,25.15,30.40,37.04,45.31,52.71,52.54,43.75,34.23,29.62,27.77,24.36
6,8.11,17.80,32.36,49.58,63.48,65.25,52.95,37.89,32.28,36.97,44.92,50.91,55.59,58.92,56.00,46.11,36.16,30.01,24.83,18.04
7,2.77,7.74,18.21,35.88,56.69,68.38,63.07,50.18,44.90,50.68,59.44,63.87,65.00,65.00,60.14,48.94,36.78,27.02,18.48,10.74
8,0.79,2.92,9.26,23.72,45.91,65.11,70.22,65.00,61.59,65.00,70.60,73.01,73.21,72.54,66.12,51.53,34.59,20.87,11.23,5.09
9,0.23,1.22,5.33,17.07,38.25,60.95,74.29,77.83,77.83,77.83,77.83,77.83,79.65,81.00,73.00,52.98,30.42,14.37,5.79,1.98
10,0.18,1.08,5.07,16.84,38.57,62.57,78.22,84.84,86.94,85.99,82.70,80.59,83.59,86.94,77.83,53.69,27.65,10.91,3.48,0.92


In [10]:
OUTPUT  = 'output/cross_section_p2.csv'
cross_section = All_voxels.to_csv(OUTPUT, index=[i for i in range(1,21)], columns = [i for i in range(1,21)])

## Check Constraint

**Beamlets Check**

In [11]:
# check beamlets value
beamlets_value = []
for i in range(60):
    beamlets_value.append(beamlets[i].solution_value())
print('Beamlets values are ')
print(np.array(beamlets_value))   

Beamlets values are 
[50.83696995 10.78436182  0.          0.          0.         15.28825378
  9.73820318  0.         24.55197282 41.75654476  0.          0.
  0.          0.          0.          0.          0.         36.93970803
  8.37168508  0.          0.          0.         14.55859408  0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.         60.12525511 11.84006107  0.
  0.          0.          0.          0.         25.30740204 52.14024753
 23.26904966 43.04011671  0.         50.81681073  0.          0.        ]


**CTV check**

In [12]:
#Every voxel receives a uniform dose of 82.8 
CTV_voxel = []
every = 0
for i in CTV_index:
    every = voxels[i].solution_value() 
    CTV_voxel.append(every)
print('CTV voxel dose(Gy)')
print(np.array(CTV_voxel))

CTV voxel dose(Gy)
[77.832      77.832      77.832      77.832      77.832      79.65405549
 80.99950322 78.22393139 84.84485359 86.94       85.99005553 82.70040972
 80.58927882 83.59034668 86.94       77.832      83.97808093 85.68808044
 86.03842057 86.94       85.69445541 83.30201586 84.60790727 86.94
 78.08292581 82.8        78.45371494 81.35106617 84.99630899 84.25615587
 82.45094977 81.64244299]


In [13]:
# the distance to 82.8
zp0 = np.array([[x.solution_value() for x in z_p]])
zn0 = np.array([[x.solution_value() for x in z_n]])
print('Positive distance to 82.8 \n',zp0)
print('\n')
print('Negative distance to 82.8 \n',zn0)

Positive distance to 82.8 
 [[4.968      4.968      4.968      4.968      4.968      3.14594451
  1.80049678 4.57606861 0.         0.         0.         0.09959028
  2.21072118 0.         0.         4.968      0.         0.
  0.         0.         0.         0.         0.         0.
  4.71707419 0.         4.34628506 1.44893383 0.         0.
  0.34905023 1.15755701]]


Negative distance to 82.8 
 [[0.         0.         0.         0.         0.         0.
  0.         0.         2.04485359 4.14       3.19005553 0.
  0.         0.79034668 4.14       0.         1.17808093 2.88808044
  3.23842057 4.14       2.89445541 0.50201586 1.80790727 4.14
  0.         0.         0.         0.         2.19630899 1.45615587
  0.         0.        ]]


In [14]:
print('maximum difference of CTV voxels values to 82.8 are ', (max(CTV_voxel)-82.8)/82.8, (82.8- min(CTV_voxel))/82.8)

maximum difference of CTV voxels values to 82.8 are  0.05000000000024973 0.0600000000001231


In [15]:
print('maximum CTV voxels value is ', max(CTV_voxel))

maximum CTV voxels value is  86.94000000002067


In [16]:
print('minimum CTV voxels value is ', min(CTV_voxel))

minimum CTV voxels value is  77.8319999999898


In [17]:
print('percentage of voxels below 79:', sum((np.array(CTV_voxel) < 79)*1) / len(CTV_voxel))

percentage of voxels below 79: 0.28125


In [18]:
print('percentage of voxels Above 86.94:', sum((np.array(CTV_voxel) > 86.94)*1) / len(CTV_voxel))

percentage of voxels Above 86.94: 0.03125


**Bladder Check**

In [19]:
# Max dose to a voxel: 81
bladder_voxel = []
every = 0
for i in bladder_index:
    every = round(voxels[i].solution_value(),4)
    bladder_voxel.append(every)
print('Bladder Voxel Doses\n',np.array(bladder_voxel))

Bladder Voxel Doses
 [29.7586 23.7243 25.1538 30.3957 37.0432 45.3054 52.7107 52.9463 37.8917
 32.282  36.9698 44.9216 50.9072 55.5854 58.92   56.0001 63.071  50.1766
 44.8972 50.6805 59.443  63.8687 65.     65.     60.1357 65.     61.5903
 65.     70.6027 73.0107 73.2146 72.54  ]


In [20]:
# if any voxel has more than 81
zb1 = np.array([[x.solution_value() for x in z_b_1]])
print('distances to 81 if voxel value is above 81\n',zb1)

distances to 81 if voxel value is above 81
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]]


In [21]:
# check if there are voxel with value more than 81
print('are there values more than 81?')
print(max(bladder_voxel) > 81)

print('how many percentage of voxel values are more than 81?')

print(round(sum((np.array(bladder_voxel) >81)*1)/len(bladder_voxel),2)*100,'%')

are there values more than 81?
False
how many percentage of voxel values are more than 81?
0.0 %


In [22]:
# average dose shouldbe <= 50
print('average dose received by bladder voxels are ',sum(bladder_voxel)/len(bladder_voxel))
print('slack variable to bring down the value to below 50: ', z_b_2.solution_value())

average dose received by bladder voxels are  52.304587500000004
slack variable to bring down the value to below 50:  2.3045869156515746


In [23]:
# at most 10% of the bladder should receive a dose > 65
y_value = np.array([t.solution_value() for t in y])
print('voxel that are larger that 65 are indicated by 1 \n',y_value)

voxel that are larger that 65 are indicated by 1 
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1.]


In [24]:
test = np.array(list(map(lambda x: 1 if x > 65.00 else 0,bladder_voxel)))
print(round(sum(test)/len(test),4), ' of the voxels are above 65')

0.125  of the voxels are above 65


In [25]:
print('voxel that are larger that 65 are indicated by 1 \n',test)

voxel that are larger that 65 are indicated by 1 
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1]


In [26]:
y_value == test

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [27]:
print('the number of constraints we have to set loose to be above 65',z_y.solution_value())

the number of constraints we have to set loose to be above 65 1.0


**Rectal Solid Check**

In [28]:
# max dose to a voxel: 79.2
Rectal_voxel = []
every = 0
for i in Rectal_index:
    every = round(voxels[i].solution_value(),8)
    Rectal_voxel.append(every)
print('rectal voxel dose values')
print(np.array(Rectal_voxel))

rectal voxel dose values
[70.0181162  72.64130625 78.5388936  79.27603036 76.88281328 76.3470604
 63.97025629 63.345402   66.47237838 67.23475016 68.40459228 71.00556969
 74.74312902 59.13419635 53.30255894 51.76896265 52.40930475 58.86879339
 68.14324089 52.29738591 42.10244858 38.66970757 40.53761218 50.80775908]


In [29]:
zr1 = np.array([[x.solution_value() for x in z_r_1]])
print('distances to 79.2 if voxel value is above 79.2\n',zr1)

distances to 79.2 if voxel value is above 79.2
 [[0.         0.         0.         0.07603036 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]]


In [30]:
# check if there's value larger than 79.2
print('are there values more than 79.2?')
print(max(Rectal_voxel) > 79.2)

print('how many percentage of voxel values are more than 79.2?')

print(round(sum((np.array(Rectal_voxel) >79.2)*1)/len(Rectal_voxel),2)*100,'%')

are there values more than 79.2?
True
how many percentage of voxel values are more than 79.2?
4.0 %


In [31]:
# average dose should be <= 40
print('average dose received by bladder voxels are ',sum(Rectal_voxel)/len(Rectal_voxel))
print('slack variable to bring down the value to below 40: ',z_r_2.solution_value())

average dose received by bladder voxels are  62.37176117500001
slack variable to bring down the value to below 40:  22.371761174754084


**Unspecified**

In [32]:
# Max dose to a voxel: 72
Un_voxel = []
every = 0
for i in Un_index:
    every = round(voxels[i].solution_value(),3)
    Un_voxel.append(every)
np.array(Un_voxel)

array([ 64.613,  46.555,  30.304,  23.189,  24.231,  25.255,  19.884,
        11.228,   5.907,   5.639,   7.81 ,  10.612,  15.415,  22.115,
        26.161,  25.249,  22.754,  21.46 ,  20.987,  21.075,  66.938,
        59.317,  44.38 ,  33.177,  30.617,  30.384,  24.51 ,  14.838,
         8.26 ,   7.177,   9.291,  12.899,  19.423,  28.254,  33.544,
        32.005,  27.438,  24.072,  22.802,  23.522,  55.336,  61.291,
        55.789,  45.894,  39.933,  36.633,  29.544,  19.371,  12.097,
        10.367,  12.587,  17.424,  25.962,  36.364,  41.366,  37.6  ,
        30.449,  25.825,  24.907,  26.176,  36.456,  50.684,  57.516,
        56.279,  51.652,  45.251,  35.257,  24.223,  17.306,  16.181,
        19.269,  25.377,  34.961,  45.06 ,  47.942,  41.316,  32.37 ,
        27.684,  27.151,  27.126,  19.189,  33.502,  47.901,  58.017,
        61.378,  56.003,  42.923,  52.536,  43.751,  34.233,  29.619,
        27.773,  24.355,   8.105,  17.805,  63.48 ,  65.248,  46.114,
        24.834,  18.

In [33]:
print('Are there voxel values more than 72?')
max(Un_voxel) > 72

Are there voxel values more than 72?


True

In [34]:
print('how many percentage of voxel values are more than 72?')

print(round(sum((np.array(Un_voxel) >72)*1)/len(Un_voxel),2)*100,'%')

how many percentage of voxel values are more than 72?
21.0 %


**Left Femur Head**

In [35]:
# max dose to be voxel 50
Left_voxel = []
every = 0
for i in Left_index:
    every = round(voxels[i].solution_value(),2)
    Left_voxel.append(every)
print('left femur head voxel values')
np.array(Left_voxel)

left femur head voxel values


array([36.16, 30.01, 48.94, 36.78, 27.02, 18.48, 51.53, 34.59, 20.87,
       11.23, 52.98, 30.42, 14.37,  5.79, 53.69, 27.65, 10.91,  3.48,
       54.73, 29.24, 12.46,  4.54, 55.85, 34.64, 18.81,  9.24, 40.  ,
       27.4 ])

In [36]:
zl1 = np.array([[x.solution_value() for x in z_l_1]])
print('distances to 79.2 if voxel value is above 50\n',zl1)

distances to 79.2 if voxel value is above 50
 [[0.         0.         0.         0.         0.         0.
  1.52893492 0.         0.         0.         2.97707026 0.
  0.         0.         3.68619995 0.         0.         0.
  4.72795855 0.         0.         0.         5.85005815 0.
  0.         0.         0.         0.        ]]


In [37]:
print('are there values more than 50?')
print(max(Left_voxel) > 50)

print('how many percentage of voxel values are more than 50?')

print(round(sum((np.array(Left_voxel) >50)*1)/len(Left_voxel),2)*100,'%')

are there values more than 50?
True
how many percentage of voxel values are more than 50?
18.0 %


In [38]:
# at most 15% of the left femur head should receive > 40
y_left_value = np.array([t.solution_value() for t in y_left])
print('voxel that are larger that 40 are indicated by 1 \n',y_left_value)

voxel that are larger that 40 are indicated by 1 
 [0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0.
 0. 0. 0. 0.]


In [39]:
test_left_femur = np.array(list(map(lambda x: 1 if x >40.00 else 0,Left_voxel)))
print(round(sum(test_left_femur)/len(test_left_femur),4), ' of the left femur voxels are above 40')

0.2143  of the left femur voxels are above 40


In [40]:
print('voxel that are larger that 40 are indicated by 1 \n',test_left_femur)

voxel that are larger that 40 are indicated by 1 
 [0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0]


In [41]:
test_left_femur == y_left_value

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [42]:
print('the number of constraints we have to set loose to be above 40',z_y_left.solution_value())

the number of constraints we have to set loose to be above 40 2.0


**Right Femur Head**

In [43]:
# max dose to a voxel: 50 
Right_voxel = []
every = 0
for i in Right_index:
    every = round(voxels[i].solution_value(),2)
    Right_voxel.append(every)
print("right femur head voxel values")
np.array(Right_voxel)

right femur head voxel values


array([17.8 , 32.36,  2.77,  7.74, 18.21, 35.88,  0.79,  2.92,  9.26,
       23.72,  0.23,  1.22,  5.33, 17.07,  0.18,  1.08,  5.07, 16.84,
        0.55,  2.31,  8.24, 23.18,  1.99,  6.12, 16.17, 36.13, 14.76,
       30.26])

In [44]:
# slack variable needed to fullfill the constraints
zrl1 = np.array([[x.solution_value() for x in z_rl_1]])
print('distances to 79.2 if voxel value is above 50\n',zrl1)

distances to 79.2 if voxel value is above 50
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]]


In [45]:
# check if there's any right voxel value that is bigger than 50
print('are there values more than 50?')
print(max(Right_voxel) > 50)

print('how many percentage of voxel values are more than 50?')

print(round(sum((np.array(Right_voxel) >50)*1)/len(Right_voxel),2)*100,'%')

are there values more than 50?
False
how many percentage of voxel values are more than 50?
0.0 %


In [46]:
# at most 15% of the right  femur head should receive > 40
y_right_value = np.array([t.solution_value() for t in y_right])
print('voxel that are larger that 40 are indicated by 1 \n',y_right_value)

voxel that are larger that 40 are indicated by 1 
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


In [47]:
test_right = np.array(list(map(lambda x: 1 if x > 40.00 else 0,Right_voxel)))
print(round(sum(test_right)/len(test_right),4), ' of the right femur voxels are above 40')

0.0  of the right femur voxels are above 40


In [48]:
print('voxel that are larger that 40 are indicated by 1 \n',test_right)

voxel that are larger that 40 are indicated by 1 
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [49]:
test_right == y_right_value

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [50]:
print('the number of constraints we have to set loose to be above 40',z_y_right.solution_value())

the number of constraints we have to set loose to be above 40 0.0
